In [89]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
import torch.optim as optim

# 1. Basic autograd example 1

In [90]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3
print(y)

# Compute gradients.
y.backward()

# get the gradients
print(x.grad.item())    # x.grad = 2 
print(w.grad.item())    # w.grad = 1 
print(b.grad.item())    # b.grad = 1 

tensor(5., grad_fn=<AddBackward0>)
2.0
1.0
1.0


# 2. Basic autograd exampled 2

In [91]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer and print out its weight
linear = nn.Linear(3, 2)
print ('w: ', linear.weight,linear.weight.shape)
print ('b: ', linear.bias,linear.bias.shape)

# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = optim.SGD(linear.parameters(), lr = 0.01)
                  
# Forward pass.
output = linear(x)
                  
# compute loss
loss = criterion(output, y)
print('loss: ', loss.item())

# backward propogate
loss.backward()

# print out gradient
print(linear.weight.grad.numpy(),linear.weight.grad.size())

# update the gradient descent
optimizer.step()
# the other way to update the weights&bias
# for i in linear.parameters():
#     i.data.subtract_(i.grad.data * 0.01)

# print new loss
output = linear(x)
loss = criterion(output,y)
print('loss: ',loss.item())

w:  Parameter containing:
tensor([[-0.0763,  0.1268, -0.0266],
        [ 0.3027,  0.1244, -0.0154]], requires_grad=True) torch.Size([2, 3])
b:  Parameter containing:
tensor([0.4635, 0.2911], requires_grad=True) torch.Size([2])
loss:  0.6007221937179565
[[-0.15569553 -0.2975867   0.34136912]
 [ 0.06994706 -0.4593457   0.15255065]] torch.Size([2, 3])
loss:  0.593173623085022


# 3. Loading data from numpy

In [92]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

# 4. Input pipeline or downloadable dataset

In [93]:
train_dataset = torchvision.datasets.CIFAR10(root='../../data/', # save folder
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print(train_dataset)
print (image.size())
print (label)

# find how many classes
print(len(train_dataset.classes))

# find how many cases
print(train_dataset.__len__())

Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../../data/
    Split: Train
    StandardTransform
Transform: ToTensor()
torch.Size([3, 32, 32])
6
10
50000


In [94]:
print(train_dataset.classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [95]:
# Data loader (this provides queues and threads in a very simple way).
batch_size = int(train_dataset.__len__() / 1000)
trainloader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=batch_size, # use dataloader to prepare data for training and testing
                                          shuffle=True)

# fetch one mini-batch from dataload

dataiter = iter(trainloader)
images, labels = dataiter.next()
print(images.shape)
print(labels.shape)

torch.Size([50, 3, 32, 32])
torch.Size([50])


# 5. Input pipeline for custom dataset!!

In [84]:
class CustomDataset(torch.utils.data.Dataset): # parent class 
    
    # the following three things are mandatory to have 
    
    # Of course each function can have added arguments. e.g. def __init__(self,*,*...)
    
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0

In [85]:
# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)

ValueError: num_samples should be a positive integer value, but got num_samples=0

# 6. Load Pre-trained model and Train for this dataset

In [96]:
# Download and load the pre-trained ResNet-18
model = torchvision.models.resnet18(pretrained=True)

# print(model)

In [97]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.001)

In [98]:
# If you want to finetune only the top layer of the model, set as below.
# for param in resnet.parameters():
#     param.requires_grad = False
    
# replace the top layer for finetuning
model.fc = nn.Linear(in_features = 512, out_features = len(train_dataset.classes), bias = True) 

# train for 3 epochs
for epoch in range(1):
    
    loss_sum = 0
    for mini_batch_i, data in enumerate(trainloader):
        
        # zero-buffer
        model.zero_grad()
        
        # get images and labels
        Images, Labels = data
        
        # forward pass
        outputs = model(Images)
        
        # loss
        loss = criterion(outputs , Labels)
        
        # backward pass
        loss.backward()
        
        # update weights
        optimizer.step()
        
        # calculate the loss
        loss_sum += loss
        if mini_batch_i % 100 == 99:
            print('now we are at epoch ',epoch,' batch ',mini_batch_i+1,' averaged loss: ', round(loss_sum.item() / 100,4))
            loss_sum = 0


now we are at epoch  0  batch  100  averaged loss:  2.418
now we are at epoch  0  batch  200  averaged loss:  2.0237
now we are at epoch  0  batch  300  averaged loss:  1.791
now we are at epoch  0  batch  400  averaged loss:  1.65
now we are at epoch  0  batch  500  averaged loss:  1.541
now we are at epoch  0  batch  600  averaged loss:  1.453
now we are at epoch  0  batch  700  averaged loss:  1.3724
now we are at epoch  0  batch  800  averaged loss:  1.328
now we are at epoch  0  batch  900  averaged loss:  1.2757
now we are at epoch  0  batch  1000  averaged loss:  1.2456


# 7. print the weights & optimizers and save the model

In [101]:
# what is state_dict - get the current value of each weights and bias as well as each optimizer
# Print model's state_dict

print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
    # print(param_tensor, "\t", net.state_dict()[param_tensor]) get values

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([64, 3, 7, 7])
bn1.weight 	 torch.Size([64])
bn1.bias 	 torch.Size([64])
bn1.running_mean 	 torch.Size([64])
bn1.running_var 	 torch.Size([64])
bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv1.weight 	 torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight 	 torch.Size([64])
layer1.0.bn1.bias 	 torch.Size([64])
layer1.0.bn1.running_mean 	 torch.Size([64])
layer1.0.bn1.running_var 	 torch.Size([64])
layer1.0.bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv2.weight 	 torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight 	 torch.Size([64])
layer1.0.bn2.bias 	 torch.Size([64])
layer1.0.bn2.running_mean 	 torch.Size([64])
layer1.0.bn2.running_var 	 torch.Size([64])
layer1.0.bn2.num_batches_tracked 	 torch.Size([])
layer1.1.conv1.weight 	 torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight 	 torch.Size([64])
layer1.1.bn1.bias 	 torch.Size([64])
layer1.1.bn1.running_mean 	 torch.Size([64])
layer1.1.bn1.running_var 	 torch.Size([64])
layer1.1.bn1.num_batc

In [102]:
# save the model
torch.save(model,'model.ckpt')

In [103]:
# only save the parameters
torch.save(model.state_dict(),'model_parameters.pth')